# Linear Dynamical Systems (LDS) in Python

Historically Linear dynamical systems (LDS) was developed independently with hidden Markov models (HMM). However, as you will find in this example, this model has the deep relationship with HMM.<br>
In this example, I also apply the same approach in hidden Markov models (HMM) example, **EM algorithm**, for Linear dynamical systems (LDS). (See [here](./01-hmm-em-algorithm.ipynb) for HMM.)

Unlike HMM, the probability for the latent (hidden) multinomial variables $ \{\mathbf{z}_{n}\} $ is not discrete (i.e, it's continuous). And it then generates the corresponding observation $ \{\mathbf{x}_{n}\} $. (See below.)
Same as HMM, the observers can see only $ \{\mathbf{x}_{n}\} $, and the model will be estimated with obervation, $ \{\mathbf{x}_{n}\} $.

![Linear Dynamical Systems](images/lds.png?raw=true)

In this model, $ p(\mathbf{z}_{n}|\mathbf{z}_{n-1}) $ is called a **transition probability**, and $ p(\mathbf{x}_{n}|\mathbf{z}_n) $ is a **emission probability**.

> In this notebook, I denote a scalar variable by normal letter (such as, $ x $), and denote a vector (incl. a matrix) by bold letter (such as, $ \mathbf{x} $).

*back to [Readme](https://github.com/tsmatz/hmm-lds-em-algorithm/)*

##  Sampling in Linear Dynamical Systems (Generate sample data)

First of all, we'll generate sample data (observations) by using the distribution of Linear Dynamical Systems (LDS).
Here I assume that both $ p(\mathbf{z}_{n}|\mathbf{z}_{n-1}) $ (transition probability) and $ p(\mathbf{x}_{n}|\mathbf{z}_n) $ (emission probability) are a Gaussian distribution as follows. (Note that **I have omitted bias terms** in this example, in order to simplify examples.)

$$ p(\mathbf{z}_{n}|\mathbf{z}_{n-1}) = \mathcal{N}(\mathbf{z}_{n}|\mathbf{A}\mathbf{z}_{n-1}, \mathbf{\Gamma}) $$

$$ p(\mathbf{x}_{n}|\mathbf{z}_{n}) = \mathcal{N}(\mathbf{x}_{n}|\mathbf{C}\mathbf{z}_{n}, \mathbf{\Sigma}) $$

In this example, I assume that $ \{ \mathbf{z}_{n} \} $ is 3 dimensional space, and $ \{ \mathbf{x}_{n} \} $ is 2 dimensional space.

For $ \{ \mathbf{x}_{n} \} $ (observation) sampling, first I'll create the latent (hidden) distribution $ \{\mathbf{z}_n\} $ with the following parameters.<br>
$ \mathbf{A} $ is a rotation matrix in any axis (x, y, and z).

$$ \mathbf{A} = \begin{bmatrix} 0.750 & 0.433 & -0.500 \\ -0.217 & 0.875 & 0.433 \\ 0.625 & -0.217 & 0.750 \end{bmatrix} $$

$$ \mathbf{\Gamma} = \begin{bmatrix} 1.5 & 0.1 & 0.0 \\ 0.1 & 2.0 & 0.3 \\ 0.0 & 0.3 & 1.0 \end{bmatrix} $$

In [1]:
import numpy as np
import math

np.random.seed(1000)  # To evaluate results

N = 2000

# rotate pi / 6 radian in any axis
A = np.matmul(
    np.matmul(
        np.array([
            [1.0,0.0,0.0],
            [0.0,math.cos(math.pi/6),math.sin(math.pi/6)],
            [0.0,-1.0*math.sin(math.pi/6),math.cos(math.pi/6)]
        ]),
        np.array([
            [math.cos(math.pi/6),0.0,-1.0*math.sin(math.pi/6)],
            [0.0,1.0,0.0],
            [math.sin(math.pi/6),0.0,math.cos(math.pi/6)]
        ])),
    np.array([
        [math.cos(math.pi/6),math.sin(math.pi/6),0.0],
        [-1.0*math.sin(math.pi/6),math.cos(math.pi/6),0.0],
        [0.0,0.0,1.0]
    ])
)

Gamma = np.array([
    [1.5, 0.1, 0.0],
    [0.1, 2.0, 0.3],
    [0.0, 0.3, 1.0]
])

Z = np.array([[23.0, 24.0, 25.0]])
for n in range(N):
    z_prev = Z[len(Z) - 1]
    mean = np.matmul(A, z_prev)
    z_post = np.random.multivariate_normal(
        mean=mean,
        cov=Gamma,
        size=1)
    Z = np.vstack((Z, z_post))
Z

array([[ 23.        ,  24.        ,  25.        ],
       [ 15.71236153,  27.90702904,  28.17441948],
       [  9.28832674,  32.27866956,  25.05656425],
       ...,
       [-32.29319196,   7.81322471,  76.74943814],
       [-59.97730124,  48.80904779,  35.1615163 ],
       [-42.51320807,  70.05011635, -21.98105043]])

Next I'll create the corresponding observation $ \{\mathbf{x}_n\} $ (2 dimensional space) with the following parameters in $ p(\mathbf{x}_{n}|\mathbf{z}_{n}) = \mathcal{N}(\mathbf{x}_{n}|\mathbf{C}\mathbf{z}_{n}, \mathbf{\Sigma}) $.

$$ \mathbf{C} = \begin{bmatrix} 1.0 & 1.0 & 0.0 \\ 0.0 & 1.0 & 1.0 \end{bmatrix} $$

$$ \mathbf{\Sigma} = \begin{bmatrix} 1.0 & 0.2 \\ 0.2 & 2.0 \end{bmatrix} $$

In [2]:
C = np.array([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
])

Sigma = np.array([
    [1.0,0.2],
    [0.2,2.0],
])

X = np.empty((0,2))
for z_n in Z:
    mean = np.matmul(C, z_n)
    x_n = np.random.multivariate_normal(
        mean=mean,
        cov=Sigma,
        size=1)
    X = np.vstack((X, x_n))
X

array([[ 47.68902127,  48.86610347],
       [ 43.94953287,  54.53438865],
       [ 41.39095419,  55.59938853],
       ...,
       [-24.57596382,  84.39783143],
       [-12.60208378,  84.9819878 ],
       [ 29.52247208,  49.04898829]])

## EM algorithm in Linear Dynamical Systems (LDS)

Now, using the given observation $ \{ \mathbf{x}_n \} $, let's try to get the optimimal parameters in LDS.

When I denote unknown parameters by $ \mathbf{\theta} $, our goal is to get the optimal parameters $ \mathbf{\theta} $ to maximize the following (1).

$$ p(\mathbf{X}|\mathbf{\theta}) = \sum_{\mathbf{Z}} p(\mathbf{X},\mathbf{Z}|\mathbf{\theta}) \;\;\;\;(1) $$

where $ \mathbf{Z} = \{\mathbf{z}_n\} $ and $ \mathbf{X} = \{\mathbf{x}_n\} $

As I have mentioned in [here](./01-hmm-em-algorithm.ipynb), it's difficult to apply [maximum likelihood estimation (MLE)](https://tsmatz.wordpress.com/2017/08/30/glm-regression-logistic-poisson-gaussian-gamma-tutorial-with-r/) for the expression (1), and I will then apply **EM algorithm** (**E**xpectation–**M**aximization algorithm) to solve unknown parameters.

In EM algorithm for LDS, we start with initial parameters $ \mathbf{\theta}^{old} $, and optimize (find) new $ \mathbf{\theta} $ to maximize the following expression (2).<br>
By repeating this operation, we can expect to reach to the likelihood parameters $ \hat{\mathbf{\theta}} $.

$$ Q(\mathbf{\theta}, \mathbf{\theta}^{old}) = \sum_{\mathbf{Z}} p(\mathbf{Z}|\mathbf{X}, \mathbf{\theta}^{old}) \ln p(\mathbf{X}, \mathbf{Z}|\mathbf{\theta}) \;\;\;\;(2) $$

> Note : For the essential idea of EM algorithm, see Chapter 9 in "[Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf?ranMID=24542&ranEAID=TnL5HPStwNw&ranSiteID=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&epi=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&irgwc=1&OCID=AID2200057_aff_7593_1243925&tduid=%28ir__vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300%29%287593%29%281243925%29%28TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ%29%28%29&irclickid=_vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300)" (Christopher M. Bishop, Microsoft)

In LDS, we use  the following parameters as $ \mathbf{\theta} = \{ \mathbf{m}_0, \mathbf{S}_0, \mathbf{A}, \mathbf{\Gamma}, \mathbf{C}, \mathbf{\Sigma} \} $.

- $ \mathbf{m}_0, \mathbf{S}_0 $ : Gaussian distribution's parameters (mean, variance) in initial latent node $ p(\mathbf{z}_0) = \mathcal{N}(\mathbf{m}_0, \mathbf{S}_0) $.
- $ \mathbf{A}, \mathbf{\Gamma} $ : parameters in transition probability $ p(\mathbf{z}_{n}|\mathbf{z}_{n-1}) = \mathcal{N}(\mathbf{z}_{n}|\mathbf{A}\mathbf{z}_{n-1}, \mathbf{\Gamma}) $.<br>
(In this example, I have omitted a bias term for simplicity.)
- $ \mathbf{C}, \mathbf{\Sigma} $ : parameters in emission probability $ p(\mathbf{x}_{n}|\mathbf{z}_{n}) = \mathcal{N}(\mathbf{x}_{n}|\mathbf{C}\mathbf{z}_{n}, \mathbf{\Sigma}) $.<br>
(In this example, I have omitted a bias term for simplicity.)

Now I denote the probability $ p(\mathbf{z}_n|\mathbf{x}_1, \ldots, \mathbf{x}_n, \mathbf{\theta}^{old}) $ by $ \alpha(z_n) = \mathcal{N}(\mathbf{z}_{n}|\mathbf{\mu}_n, \mathbf{V}_n) $.<br>
It's then known that $ \mathbf{\mu}_n  $ and $ \mathbf{V}_n $ can be recursively given by $ \mathbf{\theta}^{old} , \mathbf{X} $ as follows.

$$ \mathbf{\mu}_n = \mathbf{A}^{old} \mathbf{\mu}_{n-1} + \mathbf{K}_{n-1} (\mathbf{x}_n - \mathbf{C}^{old} \mathbf{A}^{old} \mathbf{\mu}_{n-1}) $$

$$ \mathbf{V}_n = (\mathbf{I} - \mathbf{K}_{n-1} \mathbf{C}^{old}) \mathbf{P}_{n-1} $$

where

$ \mathbf{P}_{n} = \mathbf{A}^{old} \mathbf{V}_{n} (\mathbf{A}^{old})^{T} + \mathbf{\Gamma}^{old} $

and

$ \mathbf{K}_n = \mathbf{P}_{n} (\mathbf{C}^{old})^T (\mathbf{C}^{old} \mathbf{P}_{n} (\mathbf{C}^{old})^T + \mathbf{\Sigma}^{old})^{-1} $

> Note : In [HMM](./01-hmm-em-algorithm.ipynb), $ \alpha(z_n) $ is defined as $ \alpha(z_n)=p(\mathbf{x}_1, \ldots, \mathbf{x}_n, \mathbf{z}_n |\mathbf{\theta}^{old}) $. As I mentioned in HMM, the series of these variables $ \alpha(z_n) $ will go to zero exponentially, when $ N $ is large. It will then eventually exceed the dynamic range of precision in computation, and we should introduce a scaling factor in each step $ n $.<br>
> In this example, $ \alpha(z_n) $ is scaled variables $ \alpha(z_n)=p(\mathbf{z}_n|\mathbf{x}_1, \ldots, \mathbf{x}_n, \mathbf{\theta}^{old}) $ and you don't need to scale. Therefore, when you monitor the value of likelihood functions, you also need to calculate scaling factors $ \{ c_n \} $ in each step. (See Chapter 13 in "[Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf?ranMID=24542&ranEAID=TnL5HPStwNw&ranSiteID=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&epi=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&irgwc=1&OCID=AID2200057_aff_7593_1243925&tduid=%28ir__vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300%29%287593%29%281243925%29%28TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ%29%28%29&irclickid=_vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300)" (Christopher M. Bishop, Microsoft) for details.)

In this recursion, the starting condition $ \mathbf{\mu}_0, \mathbf{V}_0 $ is :

$$ \mathbf{\mu}_0 = \mathbf{m}_0^{old} + \mathbf{K}_0 (\mathbf{x}_0 - \mathbf{C}^{old} \mathbf{m}_0^{old}) $$

$$ \mathbf{V}_0 = (\mathbf{I} - \mathbf{S}_0^{old} (\mathbf{C}^{old})^T (\mathbf{C}^{old} \mathbf{S}_0^{old} (\mathbf{C}^{old})^T + \mathbf{\Sigma}^{old})^{-1} \mathbf{C}^{old}) \mathbf{S}_0^{old} $$

> Note : When we assume $ \mathbf{P}_{-1} = \mathbf{S}_0 $, then $ \mathbf{V}_0 $ is denoted by $ (\mathbf{I} - \mathbf{K}_{-1} \mathbf{C}^{old}) \mathbf{P}_{-1} $. As you can find, it is consistent with the previous expression.<br>
> This $ \mathbf{K} $ is called a Kalman gain matrix.<br>
> Same as $ \mathbf{V}_0 $ and $ \mathbf{S}_0 $, $ \mathbf{\mu}_0 $ is also related with $ \mathbf{m}_0 $.

Now I also denote the probability $ p(\mathbf{z}_n|\mathbf{X},\mathbf{\theta}^{old}) $ by $ \gamma(z_n) = \mathcal{N}(\mathbf{z}_{n}|\hat{\mathbf{\mu}}_n, \hat{\mathbf{V}}_n) $.

Once you've done previous forward recursion for $ \{ \mathbf{\mu}_n \} $ and $ \{ \mathbf{V}_n \} $, run the following backward recursion and then get $ \{ \hat{\mathbf{\mu}}_n \}  $ and $ \{ \hat{\mathbf{V}}_n \} $ by using $ \mathbf{\theta}^{old} , \mathbf{X} $, and previous $ \{ \mathbf{\mu}_n \}, \{ \mathbf{V}_n \} $.

$$ \hat{\mathbf{\mu}}_n = \mathbf{\mu}_n + \mathbf{J}_n (\hat{\mathbf{\mu}}_{n+1} - \mathbf{A}^{old} \mathbf{\mu}_n) $$

$$ \hat{\mathbf{V}}_n = \mathbf{V}_n + \mathbf{J}_n (\hat{\mathbf{V}}_{n+1} - \mathbf{P}_n) \mathbf{J}_n^T $$

where $ \mathbf{J}_n = \mathbf{V}_n (\mathbf{A}^{old})^T (\mathbf{P}_n)^{-1} $.

Once you have got these variables, now you can get the following expectation values.

$$ \mathbb{E}[\mathbf{z}_n] = \hat{\mathbf{\mu}}_n $$

$$ \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^T] = \hat{\mathbf{V}}_n \mathbf{J}_{n-1}^{T} + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_{n-1}^T $$

$$ \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] = \hat{\mathbf{V}}_n + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_n^T $$

> Note : $ \mathbb{E}[\mathbf{z}_{n-1} \mathbf{z}_{n}^T] = \left( \hat{\mathbf{V}}_n \mathbf{J}_{n-1}^{T} + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_{n-1}^T \right)^T $

With these expectation values, you can get the optimal $ \mathbf{\theta} = \{ \mathbf{m}_0, \mathbf{S}_0, \mathbf{A}, \mathbf{\Gamma}, \mathbf{C}, \mathbf{\Sigma} \} $ to maximize (2) as follows.

$$ \mathbf{m}_0^{new} = \mathbb{E}[\mathbf{z}_0] $$

$$ \mathbf{S}_0^{new} = \mathbb{E}[\mathbf{z}_0 \mathbf{z}_0^T] - \mathbb{E}[\mathbf{z}_0]\mathbb{E}[\mathbf{z}_0^T] $$

$$ \mathbf{A}^{new} = \left( \sum_{n=1}^{N-1} \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^T] \right) \left( \sum_{n=0}^{N-2} \mathbb{E}[\mathbf{z}_{n} \mathbf{z}_{n}^T] \right)^{-1} $$

$$ \mathbf{\Gamma}^{new} = \frac{1}{N - 1} \sum_{n=1}^{N-1} \left\{ \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] - \mathbf{A}^{new} \mathbb{E}[\mathbf{z}_{n-1} \mathbf{z}_n^T] - \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^{T}](\mathbf{A}^{new})^T + \mathbf{A}^{new} \mathbb{E}[\mathbf{z}_{n-1}\mathbf{z}_{n-1}^T](\mathbf{A}^{new})^T \right\} $$

$$ \mathbf{C}^{new} = \left( \sum_{n=0}^{N-1} \mathbf{x}_n \mathbb{E}[\mathbf{z}_n]^T \right) \left( \sum_{n=0}^{N-1} \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] \right)^{-1} $$

$$ \mathbf{\Sigma}^{new} = \frac{1}{N} \sum_{n=0}^{N-1} \left\{ \mathbf{x}_n \mathbf{x}_n^T - \mathbf{C}^{new} \mathbb{E}[\mathbf{z}_n] \mathbf{x}_n^T - \mathbf{x}_n \mathbb{E}[\mathbf{z}_n^T](\mathbf{C}^{new})^T + \mathbf{C}^{new} \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] (\mathbf{C}^{new})^T \right\} $$

You should repeat this process by replacing $ \mathbf{\theta}^{old} $ with new $ \mathbf{\theta} $, and you will eventually get the optimal results $ \hat{\mathbf{\theta}} $.

> Note : For these properties, please refer Chapter 13 in "[Pattern Recognition and Machine Learning](https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf?ranMID=24542&ranEAID=TnL5HPStwNw&ranSiteID=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&epi=TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ&irgwc=1&OCID=AID2200057_aff_7593_1243925&tduid=%28ir__vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300%29%287593%29%281243925%29%28TnL5HPStwNw-g4zE85KQgCXaCQfYBhtuFQ%29%28%29&irclickid=_vhvv9m6caokf6nb62oprh029if2xo0rux3ga300300)" (Christopher M. Bishop, Microsoft).

## Apply algorithm in Python

## 0. Prerequisites

In [ ]:
!pip3 install numpy
!pip3 install scipy

## 1. Initialize parameters

First, initialize $ \mathbf{\theta} = \{ \mathbf{m}_0, \mathbf{S}_0, \mathbf{A}, \mathbf{\Gamma}, \mathbf{C}, \mathbf{\Sigma} \} $ as follows.<br>
In this example, I set the primitive fixed values.

- $ \mathbf{m}_0 = (10.0, 10.0, 10.0) $
- $ \mathbf{S}_0 = \begin{bmatrix} 1.0 & 0.5 & 0.5 \\ 0.5 & 1.0 & 0.5 \\ 0.5 & 0.5 & 1.0 \end{bmatrix} $
- $ \mathbf{A} = \begin{bmatrix} 1.0 & 1.1 & 1.2 \\ 1.3 & 1.4 & 1.5 \\ 1.6 & 1.7 & 1.8 \end{bmatrix} $
- $ \mathbf{\Gamma} = \begin{bmatrix} 1.0 & 0.5 & 0.5 \\ 0.5 & 1.0 & 0.5 \\ 0.5 & 0.5 & 1.0 \end{bmatrix} $
- $ \mathbf{C} = \begin{bmatrix} 1.0 & 1.0 & 1.0 \\ 1.0 & 1.0 & 1.0 \end{bmatrix} $
- $ \mathbf{\Sigma} = \begin{bmatrix} 1.0 & 0.5 \\ 0.5 & 1.0 \end{bmatrix} $

In [3]:
# Initialize parameters
class theta:
    m0 = np.empty((0,3))
    S0 = np.empty((0,3,3))
    A = np.empty((0,3,3))
    Gamma = np.empty((0,3,3))
    C = np.empty((0,2,3))
    Sigma = np.empty((0,2,2))

    def __init__(self, m0, S0, A, Gamma, C, Sigma):
        self.m0 = m0
        self.S0 = S0
        self.A = A
        self.Gamma = Gamma
        self.C = C
        self.Sigma = Sigma

theta_old = theta(
    m0=np.array([10.0, 10.0, 10.0]),
    S0=np.array([
        [1.0, 0.5, 0.5],
        [0.5, 1.0, 0.5],
        [0.5, 0.5, 1.0]
    ]),
    A=np.array([
        [1.0, 1.1, 1.2],
        [1.3, 1.4, 1.5],
        [1.6, 1.7, 1.8]
    ]),
    Gamma=np.array([
        [1.0, 0.5, 0.5],
        [0.5, 1.0, 0.5],
        [0.5, 0.5, 1.0]
    ]),
    C=np.array([
        [1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0],
    ]),
    Sigma=np.array([
        [1.0, 0.5],
        [0.5, 1.0]
    ])
)

## 2. Get $ \{ \mathbf{\mu}_n \} $ and $ \{ \mathbf{V}_n \} $

Now I set the starting condition, $ \mathbf{V}_0 $ as follows. :

$$ \mathbf{V}_0 = (\mathbf{I} - \mathbf{S}_0^{old} (\mathbf{C}^{old})^T (\mathbf{C}^{old} \mathbf{S}_0^{old} (\mathbf{C}^{old})^T + \mathbf{\Sigma}^{old})^{-1} \mathbf{C}^{old}) \mathbf{S}_0^{old} $$

where $ \mathbf{P}_{n} = \mathbf{A}^{old} \mathbf{V}_{n} (\mathbf{A}^{old})^{T} + \mathbf{\Gamma}^{old} $ and $ \mathbf{K}_n = \mathbf{P}_{n} (\mathbf{C}^{old})^T (\mathbf{C}^{old} \mathbf{P}_{n} (\mathbf{C}^{old})^T + \mathbf{\Sigma}^{old})^{-1} $

As I mentioned above, this is equivalent to :

$$ \mathbf{V}_0 = (\mathbf{I} - \mathbf{K}_{-1} \mathbf{C}^{old}) \mathbf{S}_0^{old} $$

where $ \mathbf{K}_{-1} = \mathbf{S}_0^{old} (\mathbf{C}^{old})^T (\mathbf{C}^{old} \mathbf{S}_0^{old} (\mathbf{C}^{old})^T + \mathbf{\Sigma}^{old})^{-1} $

And we can recursively obtain all $ \{ \mathbf{V}_n \} $ as follows.

$$ \mathbf{V}_n = (\mathbf{I} - \mathbf{K}_{n-1} \mathbf{C}^{old}) \mathbf{P}_{n-1} $$

In [4]:
from scipy.stats import multivariate_normal

def P(V_n):
    res = np.matmul(theta_old.A, V_n)
    res = np.matmul(res, theta_old.A.transpose())
    res = res + theta_old.Gamma
    return res

def K(P_n):
    res = np.matmul(P_n, theta_old.C.transpose())
    inv = np.matmul(theta_old.C, P_n)
    inv = np.matmul(inv, theta_old.C.transpose())
    inv = inv + theta_old.Sigma
    inv = np.linalg.inv(inv)
    res = np.matmul(res, inv)
    return res

def get_V():
    V = np.empty((0,3,3))
    I = np.array([[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]])

    # Get initial V_0
    K_minus1 = K(theta_old.S0)
    V_0 = np.matmul(
        np.subtract(I, np.matmul(K_minus1, theta_old.C)),
        theta_old.S0)
    V = np.vstack((V, [V_0]))

    # Get all elements recursively
    for n in range(1, N):
        P_n_minus1 = P(V[n-1])
        V_n = np.matmul(
            np.subtract(I, np.matmul(K(P_n_minus1),theta_old.C)),
            P_n_minus1)
        V = np.vstack((V, [V_n]))

    return V

I also set the starting condition, $ \mathbf{\mu}_0 $ as follows. :

$$ \mathbf{\mu}_0 = \mathbf{m}_0^{old} + \mathbf{K}_0 (\mathbf{x}_0 - \mathbf{C}^{old} \mathbf{m}_0^{old}) $$

And we can recursively obtain all $ \{ \mathbf{\mu}_n \} $ as follows.

$$ \mathbf{\mu}_n = \mathbf{A}^{old} \mathbf{\mu}_{n-1} + \mathbf{K}_{n-1} (\mathbf{x}_n - \mathbf{C}^{old} \mathbf{A}^{old} \mathbf{\mu}_{n-1}) $$

In [5]:
def get_mu(V):
    mu = np.empty((0,3))

    # Get initial mu_0
    P_0 = P(V[0])
    K_0 = K(P_0)
    theta_old_m0_T = np.array([theta_old.m0]).transpose()
    X_0_T = np.array([X[0]]).transpose()
    mu_0_T = np.add(
        theta_old_m0_T,
        np.matmul(
            K_0,
            np.subtract(
                X_0_T,
                np.matmul(theta_old.C,theta_old_m0_T)
            )
        )
    )
    mu_0 = np.squeeze(mu_0_T.transpose())
    mu = np.vstack((mu, mu_0))

    # Get all elements recursively
    for n in range(1, N):
        P_n_minus1 = P(V[n-1])
        K_n_minus1 = K(P_n_minus1)
        mu_n_minus1_T = np.array([mu[n-1]]).transpose()
        mu_n_former_T = np.matmul(theta_old.A, mu_n_minus1_T)
        X_n_T = np.array([X[n]]).transpose()
        sub_n_T = np.subtract(
            X_n_T,
            np.matmul(
                np.matmul(theta_old.C,theta_old.A),
                mu_n_minus1_T
            )
        )
        mu_n_latter_T = np.matmul(
            K_n_minus1,
            sub_n_T)
        mu_n_T = np.add(mu_n_former_T, mu_n_latter_T)
        mu_n = np.squeeze(mu_n_T.transpose())
        mu = np.vstack((mu, mu_n))

    return mu

## 3. Get $ \{ \hat{\mathbf{\mu}}_n \} $ and $ \{ \hat{\mathbf{V}}_n \} $

Now we obtain $ \{ \hat{\mathbf{\mu}}_n \} $ and $ \{ \hat{\mathbf{V}}_n \} $ by running backward recursion with $ \{ \mathbf{\mu}_n \} $ and $ \{ \mathbf{V}_n \} $.

First we get $ \{ \mathbf{V}_n \} $ as follows.

$$ \hat{\mathbf{V}}_n = \mathbf{V}_n + \mathbf{J}_n (\hat{\mathbf{V}}_{n+1} - \mathbf{P}_n) \mathbf{J}_n^T $$

where $ \mathbf{J}_n = \mathbf{V}_n (\mathbf{A}^{old})^T (\mathbf{P}_n)^{-1} $

In [6]:
def J(V_n, P_n):
    return np.matmul(
        np.matmul(V_n,theta_old.A.transpose()),
        np.linalg.inv(P_n)
    )

def get_V_hat(V):
    V_hat_rev = np.empty((0,3,3))
    V_hat_rev = np.vstack((V_hat_rev, [V[N-1]]))

    for n in range(1, N):
        V_n = V[N-n-1]
        P_n = P(V_n)
        J_n = J(V_n, P_n)
        V_hat_n = np.add(
            V_n,
            np.matmul(
                np.matmul(
                    J_n,
                    np.subtract(V_hat_rev[n-1],P_n)
                ),
                J_n.transpose()
            )
        )
        V_hat_rev = np.vstack((V_hat_rev, [V_hat_n]))

    # Reverse results
    V_hat = np.flip(V_hat_rev, axis=0)

    return V_hat

Next we also get $ \{ \hat{\mathbf{\mu}}_n \} $ as follows.

$$ \hat{\mathbf{\mu}}_n = \mathbf{\mu}_n + \mathbf{J}_n (\hat{\mathbf{\mu}}_{n+1} - \mathbf{A}^{old} \mathbf{\mu}_n) $$

In [7]:
def get_mu_hat(mu, V):
    mu_hat_rev = np.empty((0,3))
    mu_hat_rev = np.vstack((mu_hat_rev, mu[N-1]))

    for n in range(1, N):
        mu_n = mu[N-n-1]
        mu_n_T = np.array([mu_n]).transpose()
        mu_hat_rev_n_minus1_T = np.array([mu_hat_rev[n-1]]).transpose()
        V_n = V[N-n-1]
        P_n = P(V_n)
        J_n = J(V_n, P_n)
        mu_hat_n_T = np.add(
            mu_n_T,
            np.matmul(
                J_n,
                np.subtract(
                    mu_hat_rev_n_minus1_T,
                    np.matmul(theta_old.A,mu_n_T)
                )
            )
        )
        mu_hat_n = np.squeeze(mu_hat_n_T.transpose())
        mu_hat_rev = np.vstack((mu_hat_rev, mu_hat_n))

    # Reverse results
    mu_hat = np.flip(mu_hat_rev, axis=0)

    return mu_hat

## 4. Get Expectations

Now we get the following array of expectations.

1. $ \mathbb{E}[\mathbf{z}_n] = \hat{\mathbf{\mu}}_n \;\; (n=0, \ldots, N-1) $
2. $ \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^T] = \hat{\mathbf{V}}_n \mathbf{J}_{n-1}^{T} + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_{n-1}^T \;\; (n=1, \ldots, N-1) $
3. $ \mathbb{E}[\mathbf{z}_{n-1} \mathbf{z}_{n}^T] = \left( \hat{\mathbf{V}}_n \mathbf{J}_{n-1}^{T} + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_{n-1}^T \right)^T \;\; (n=1, \ldots, N-1) $
4. $ \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] = \hat{\mathbf{V}}_n + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_n^T \;\; (n=0, \ldots, N-1) $

### (1) $ \mathbb{E}[\mathbf{z}_n] = \hat{\mathbf{\mu}}_n \;\; (n=0, \ldots, N-1)$

In [8]:
def get_E1(mu_hat):
    return mu_hat

### (2) $ \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^T] = \hat{\mathbf{V}}_n \mathbf{J}_{n-1}^{T} + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_{n-1}^T  \;\; (n=1, \ldots, N-1)$

### (3) $ \mathbb{E}[\mathbf{z}_{n-1} \mathbf{z}_{n}^T] = \left( \hat{\mathbf{V}}_n \mathbf{J}_{n-1}^{T} + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_{n-1}^T \right)^T \;\; (n=1, \ldots, N-1) $

In [9]:
def get_E2_E3(V, mu_hat, V_hat):
    E2 = np.empty((0,3,3))
    E3 = np.empty((0,3,3))
    for n in range(1,N):
        P_n_minus1 = P(V[n-1])
        J_n_minus1 = J(V[n-1], P_n_minus1)
        mu_hat_n_T = np.array([mu_hat[n]]).transpose()
        mu_hat_n_minus1 = np.array([mu_hat[n-1]])
        E2_n = np.add(
            np.matmul(
                V_hat[n],
                J_n_minus1.transpose()
            ),
            np.matmul(
                mu_hat_n_T,
                mu_hat_n_minus1
            )
        )
        E2 = np.vstack((E2, [E2_n]))
        E3_n = E2_n.transpose()
        E3 = np.vstack((E3, [E3_n]))
    return E2, E3

### (4) $ \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] = \hat{\mathbf{V}}_n + \hat{\mathbf{\mu}}_n \hat{\mathbf{\mu}}_n^T \;\; (n=0, \ldots, N-1) $

In [10]:
def get_E4(mu_hat, V_hat):
    E4 = np.empty((0,3,3))
    for n in range(N):
        mu_hat_n = np.array([mu_hat[n]])
        mu_hat_n_T = mu_hat_n.transpose()
        E4_n = np.add(
            V_hat[n],
            np.matmul(mu_hat_n_T, mu_hat_n)
        )
        E4 = np.vstack((E4, [E4_n]))
    return E4

## 5. Get new (optimal) parameters $ \mathbf{\theta} $

Finally, get new $ \mathbf{\theta} = \{ \mathbf{m}_0, \mathbf{S}_0, \mathbf{A}, \mathbf{\Gamma}, \mathbf{C}, \mathbf{\Sigma} \} $ using previous E1, E2, E3, and E4.

First, $ \mathbf{m}_0 $ is given as follows.

$$ \mathbf{m}_0^{new} = \mathbb{E}[\mathbf{z}_0] $$

In [11]:
def get_m0_new(E1):
    return E1[0]

$ \mathbf{S}_0 $ is given as follows.

$$ \mathbf{S}_0^{new} = \mathbb{E}[\mathbf{z}_0 \mathbf{z}_0^T] - \mathbb{E}[\mathbf{z}_0]\mathbb{E}[\mathbf{z}_0^T] $$

In [12]:
def get_S0_new(E1, E4):
    E1_0 = np.array([E1[0]])
    E1_0_T = E1_0.transpose()
    return np.subtract(
        E4[0],
        np.matmul(E1_0_T, E1_0)
    )

$ \mathbf{A} $ is given as follows.

$$ \mathbf{A}^{new} = \left( \sum_{n=1}^{N-1} \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^T] \right) \left( \sum_{n=0}^{N-2} \mathbb{E}[\mathbf{z}_{n} \mathbf{z}_{n}^T] \right)^{-1} $$

In [13]:
def get_A_new(E2, E4):
    return np.matmul(
        np.sum(E2, axis=0),
        np.linalg.inv(np.sum(E4[:N-1], axis=0))
    )

$ \mathbf{\Gamma} $ is given as follows.

$$ \mathbf{\Gamma}^{new} = \frac{1}{N - 1} \sum_{n=1}^{N-1} \left\{ \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] - \mathbf{A}^{new} \mathbb{E}[\mathbf{z}_{n-1} \mathbf{z}_n^T] - \mathbb{E}[\mathbf{z}_n \mathbf{z}_{n-1}^{T}](\mathbf{A}^{new})^T + \mathbf{A}^{new} \mathbb{E}[\mathbf{z}_{n-1}\mathbf{z}_{n-1}^T](\mathbf{A}^{new})^T \right\} $$

In [14]:
def get_Gamma_new(E2, E3, E4, A_new):
    elems = np.empty((0,3,3))
    for n in range(1, N):
        elems_n = np.add(
            np.subtract(
                np.subtract(
                    E4[n],
                    np.matmul(
                        A_new,
                        E3[n-1]
                    )
                ),
                np.matmul(
                    E2[n-1],
                    A_new.transpose()
                )
            ),
            np.matmul(
                np.matmul(
                    A_new,
                    E4[n-1]
                ),
                A_new.transpose()
            )
        )
        elems = np.vstack((elems, [elems_n]))
    return np.sum(elems, axis=0) / (N-1)

$ \mathbf{C} $ is given as follows.

$$ \mathbf{C}^{new} = \left( \sum_{n=0}^{N-1} \mathbf{x}_n \mathbb{E}[\mathbf{z}_n]^T \right) \left( \sum_{n=0}^{N-1} \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] \right)^{-1} $$

In [15]:
def get_C_new_former(E1):
    elems = np.empty((0,2,3))
    for n in range(N):
        x_n_T = np.array([X[n]]).transpose()
        E1_n = np.array([E1[n]])
        elems_n = np.matmul(x_n_T, E1_n)
        elems = np.vstack((elems, [elems_n]))
    return np.sum(elems, axis=0)

def get_C_new(E1, E4):
    return np.matmul(
        get_C_new_former(E1),
        np.linalg.inv(np.sum(E4, axis=0))
    )

$ \mathbf{\Sigma} $ is given as follows.

$$ \mathbf{\Sigma}^{new} = \frac{1}{N} \sum_{n=0}^{N-1} \left\{ \mathbf{x}_n \mathbf{x}_n^T - \mathbf{C}^{new} \mathbb{E}[\mathbf{z}_n] \mathbf{x}_n^T - \mathbf{x}_n \mathbb{E}[\mathbf{z}_n^T](\mathbf{C}^{new})^T + \mathbf{C}^{new} \mathbb{E}[\mathbf{z}_n \mathbf{z}_n^T] (\mathbf{C}^{new})^T \right\} $$

In [16]:
def get_Sigma_new(E1, E4, C_new):
    elems = np.empty((0,2,2))
    for n in range(N):
        x_n = np.array([X[n]])
        x_n_T = x_n.transpose()
        E1_n = np.array([E1[n]])
        E1_n_T = E1_n.transpose()
        elem_n = np.add(
            np.subtract(
                np.subtract(
                    np.matmul(x_n_T, x_n),
                    np.matmul(
                        np.matmul(
                            C_new,
                            E1_n_T
                        ),
                        x_n
                    )
                ),
                np.matmul(
                    np.matmul(
                        x_n_T,
                        E1_n
                    ),
                    C_new.transpose()
                )
            ),
            np.matmul(
                np.matmul(
                    C_new,
                    E4[n]
                ),
                C_new.transpose()
            )
        )
        elems = np.vstack((elems, [elem_n]))
    return np.sum(elems, axis=0) / N

## 5. Run algorithm

In [17]:
for loop in range(100):
    print("Running iteration {} ...".format(loop + 1), end="\r")
    # Get mu and V
    V = get_V()
    mu = get_mu(V)
    # Get mu_hat and V_hat
    V_hat = get_V_hat(V)
    mu_hat = get_mu_hat(mu, V)
    # Get expectation values
    E1 = get_E1(mu_hat)
    E2, E3 = get_E2_E3(V, mu_hat, V_hat)
    E4 = get_E4(mu_hat, V_hat)
    # Get optimized new parameters
    m0_new = get_m0_new(E1)
    S0_new = get_S0_new(E1, E4)
    A_new = get_A_new(E2, E4)
    Gamma_new = get_Gamma_new(E2, E3, E4, A_new)
    C_new = get_C_new(E1, E4)
    Sigma_new = get_Sigma_new(E1, E4, C_new)
    # Replace theta and repeat
    theta_old.m0 = m0_new
    theta_old.S0 = S0_new
    theta_old.A = A_new
    theta_old.Gamma = Gamma_new
    theta_old.C = C_new
    theta_old.Sigma = Sigma_new

print("\nDone")

Running iteration 100 ...
Done


Here is the estimated results for parameters.

I note that the result won't be unique (depending on scaling, rotation, etc) and this result is then one of such locally maximized results.<br>
For your reference, I show you the simulated points $ \{ \mathbf{x}_n \} $ (first 5 points in sequence) compared with observations in below.

In [18]:
print("m0")
print(m0_new)
print("S0")
print(S0_new)
print("A")
print(A_new)
print("Gamma")
print(Gamma_new)
print("C")
print(C_new)
print("Sigma")
print(Sigma_new)

m0
[ 2.57496192  6.3276932  11.234217  ]
S0
[[5.24798097e-05 2.79583705e-05 8.54545730e-05]
 [2.79583705e-05 2.07409258e-04 1.78504551e-04]
 [8.54545730e-05 1.78504551e-04 3.76386362e-04]]
A
[[ 0.78563464 -0.54698622  0.37223684]
 [ 0.89487513  0.87198508 -0.1177087 ]
 [ 0.10580215  0.43482559  0.71643318]]
Gamma
[[0.00832995 0.02118287 0.03410333]
 [0.02118287 0.06644179 0.10506952]
 [0.03410333 0.10506952 0.17151925]]
C
[[-15.90921342  -3.14027935   9.66084486]
 [ 19.55862629   7.56719346  -4.3960726 ]]
Sigma
[[0.96929266 0.04601912]
 [0.04601912 1.87507473]]


In [19]:
# Show simulated results

N_sim = 4

Z_sim = np.array([m0_new])
for n in range(N_sim):
    z_prev = Z_sim[len(Z_sim) - 1]
    mean = np.matmul(A_new, z_prev)
    z_post = np.random.multivariate_normal(
        mean=mean,
        cov=Gamma_new,
        size=1)
    Z_sim = np.vstack((Z_sim, z_post))
X_sim = np.empty((0,2))
for z_n in Z_sim:
    mean = np.matmul(C_new, z_n)
    x_n = np.random.multivariate_normal(
        mean=mean,
        cov=Sigma_new,
        size=1)
    X_sim = np.vstack((X_sim, x_n))
print("***** Observation *****")
print(X[:5])
print("***** Simulated Result *****")
print(X_sim)

***** Observation *****
[[47.68902127 48.86610347]
 [43.94953287 54.53438865]
 [41.39095419 55.59938853]
 [45.94610067 55.73904685]
 [53.12714417 48.54030761]]
***** Simulated Result *****
[[47.49019418 47.29451052]
 [44.14951187 57.2202532 ]
 [42.05716531 56.68219724]
 [43.07225681 55.11272219]
 [52.83202734 51.55821952]]
